Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import time
from dotenv import load_dotenv
import logging as log
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager

In [2]:
def check_run_status(client, thread_id: str, run_id: str, n_tries: int, wait_time):
    
    ## Wait until status is completed
    for i in range(0, n_tries):

        # Retrieve the latest run
        run_retrieve = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )

        # Get the run status
        run_status = run_retrieve.status

        # Check run status
        if run_status == 'completed':
            print('Run is completed')
            return f'Run is completed (run_id: {run_id}, thread_id {thread_id})'
        elif run_status == 'in_progress':
            print('Run is in progress')
            pass
        elif run_status == 'queued':
            print('Run is queued')
            pass
        elif run_status == 'cancelling':
            print('Run is cancelling')
            pass
        elif run_status == 'cancelled':
            raise ValueError(f'Error: run is cancelled (run_id: {run_id}, thread_id {thread_id})')
        elif run_status == 'failed':
            raise ValueError(f'Error: run has failed (run_id: {run_id}, thread_id {thread_id})')
        elif run_status == 'expired':
            raise ValueError(f'Error: run has expired (run_id: {run_id}, thread_id {thread_id})')
        elif run_status == 'requires_action':
            print('Action is required')
            return f'Action required (run_id: {run_id}, thread_id {thread_id})'

        # Sleep to give time for the run to process
        time.sleep(wait_time)

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Set constants for loops
NUM_TRIES = 12
TIME_SLEEP = 5 # in seconds

# load_dotenv('config.env')
# OPEN_API_KEY = os.getenv('OPEN_API_KEY')

# List of JSON files
# json_files = [('income_stmt', 'df_income_stmt_columns.json'), ('cashflow', 'df_cashflow_columns.json'), ('stocks', 'df_stocks_columns.json')]

# Load schemas from JSON file
with open('dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

with open('config/config.json', 'r') as f:
    dic_config = json.load(f)
    OPEN_API_KEY = dic_config['OPEN_API_KEY']
    FIN_ANALYST_ID = dic_config['fin_analyst_assistant']

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

client = OpenAI(api_key=OPEN_API_KEY)

### Stocks Data

In [5]:
stocks_list = ['MSFT', 'SHOP', 'AMD', 'NVDA', 'AAPL', 'OKTA', 'SHOP', 'AMAT', 'TTWO', 'SNOW', 'SOXX', 'BA', 'DOCN', 'MCD', 'LULU', 'CSCO', 'ORCL', 'AMZN', 'ASML']

yf_handler = YFHandler(stock_list=stocks_list, schemas=schemas)

df_stocks = yf_handler.import_stocks()
df_cashflow = yf_handler.import_cashflow()
df_income_stmt = yf_handler.import_income_stmt()

# Write to CSV
# Technically this will be done by the FileHandler but just to be safe
df_stocks.to_csv('openai_upload_files/df_stocks.csv', index=False)
df_cashflow.to_csv('openai_upload_files/df_cashflow.csv', index=False)
df_income_stmt.to_csv('openai_upload_files/df_income_stmt.csv', index=False)

file_stocks = FileHandler(
    df=df_stocks,
    dic_file=dic_files,
    file_name='df_stocks.csv',
    file_path=FILE_PATH,
    client=client
)

file_cashflow = FileHandler(
    df=df_cashflow,
    dic_file=dic_files,
    file_name='df_cashflow.csv',
    file_path=FILE_PATH,
    client=client
)

file_income_stmt = FileHandler(
    df=df_income_stmt,
    dic_file=dic_files,
    file_name='df_income_stmt.csv',
    file_path=FILE_PATH,
    client=client
)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longe

In [6]:
# Update files
file_stocks.update_openai_file(dic_file=dic_files, dic_file_name=OPENAI_DIC_FILE_NAME, dic_file_path=FILE_PATH)
file_cashflow.update_openai_file(dic_file=dic_files, dic_file_name=OPENAI_DIC_FILE_NAME, dic_file_path=FILE_PATH)
file_income_stmt.update_openai_file(dic_file=dic_files, dic_file_name=OPENAI_DIC_FILE_NAME, dic_file_path=FILE_PATH)

file name: df_stocks.csv, file id: file-6l5H1TqYaKCtnCTJeiCNfSjA has been deleted
file name: df_stocks.csv is uploaded, new file id: file-NcAYobSXtKGKAJMylEsaGF2v
openai_upload_files/openai_files.json file has been updated
file name: df_cashflow.csv, file id: file-6PfALiBYEIgDxzIAj6gtXLjC has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-Tm3qkLMs00Lop4LPDJKGKMmi
openai_upload_files/openai_files.json file has been updated
file name: df_income_stmt.csv, file id: file-M7S4zXPiQb8bsdww3M12lYut has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-Z3eZzIA2VaImoSlLLJeGSY2O
openai_upload_files/openai_files.json file has been updated


In [3]:
with open('config/assistants.json', 'r') as f:
    dic_assistants= json.load(f)

In [15]:
# Assistant id asst_I4lFSi5mtU5OYoeivxOc4piG

assistant = client.beta.assistants.update(
    assistant_id=dic_assistants['fin_analyst']['id'], 
    instructions=dic_assistants['fin_analyst']['instructions'],
    model=dic_assistants['fin_analyst']['model'],
    tools=dic_assistants['fin_analyst']['tools'],
    tool_resources={
        'code_interpreter': {
            'file_ids': [file_stocks.file_id]
        }
    }
)
# file_stocks.id

In [22]:
# # Set dictionary for tracking messages
# dic_messages = {}

# ## Create thread
# thread = client.beta.threads.create(
#     messages=[
#         {
#             'role': 'user',
#             'content': 'Can you explain to me what the file I have attached is.',
#             'attachments': [
#                 {
#                     'file_id': file_stocks.file_id,
#                     'tools': [{'type': 'code_interpreter'}]
#                 }
#             ]
#         }
#     ]
# )

# # Get thread id
# thread_id = thread.id

# # Record first message
# dic_messages['0'] = get_last_message(client=client, thread_id=thread_id)

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=FIN_ANALYST_ID,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

KeyError: '0'

In [7]:
## Create thread
thread = client.beta.threads.create(
    messages=[
        {
            'role': 'user',
            'content': 'Can you explain to me what the file I have attached is.',
            'attachments': [
                {
                    'file_id': file_stocks.file_id,
                    'tools': [{'type': 'code_interpreter'}]
                }
            ]
        }
    ]
)

# Get thread id
thread_id = thread.id

# Set ThreadManager to easily track messages in a single thread
thread_fin_analyst = ThreadManager(client=client, thread_id=thread_id)

# # Run assistants
# # Run should use stream now instaed of create
# run = client.beta.threads.runs.create(
#     thread_id = thread.id,
#     assistant_id=FIN_ANALYST_ID
# )

# run_id = run.id

# NUM_TRIES = 10
# TIME_SLEEP = 5

# check_run_status = check_run_status(client=client, thread_id=thread_id, run_id=run_id, wait_time=TIME_SLEEP, n_tries=NUM_TRIES)

# # final step is to delete the thread
# # client.beta.threads.delete(thread_id=THREAD_ID)

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=FIN_ANALYST_ID,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

thread_fin_analyst.get_last_message()


assistant > code_interpreter

import pandas as pd

# Load and inspect the first few rows of the file
file_path = '/mnt/data/file-NcAYobSXtKGKAJMylEsaGF2v'
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe
df.head()
assistant > It seems that there is an issue with determining the format of the Excel file. Let's try specifying the engine explicitly to see if that helps. I will also load the first few rows of the file to check its content.# Retry reading the file with an explicit engine
df = pd.read_excel(file_path, engine='openpyxl')

# Display the first few rows of the dataframe
df.head()
assistant > It appears that the file is not in a valid Excel format or is corrupted, as it can't be read by typical Excel file readers. This could be due to several reasons; for example, the file might have a different format than expected or it could be corrupted.

Let's inspect the other uploaded file to see if that one is also problematic or if it can be read successfully

{'message_id': 'msg_8GLQV8vCXX85ca1AiUwkpzEB',
 'assistant_id': 'asst_Mqf9cO1sDTOd4UMYwcFfIQrA',
 'created_at': 1722495443,
 'file_ids': [],
 'role': 'assistant',
 'run_id': 'run_1w2TRxE4W1VmrsBX57tJbzUT',
 'message_text': 'Both of the uploaded files appear to have issues with being read as valid Excel files. This could indicate that they are either corrupted or not actually in Excel format despite having a ".xlsx" extension.\n\nTo proceed, could you please verify the contents of the files on your end? Ensure they are indeed in Excel format. If they are intended to be different types of files (e.g., CSV, text documents, etc.), please let me know or re-upload the correct files. Alternatively, you can also try opening them with Excel or a similar program to check if they are accessible.'}

In [18]:
# Set ThreadManager to easily track messages in a single thread
thread
# client.beta.threads.delete(thread_id=thread_id)

AttributeError: 'Thread' object has no attribute 'messages'